In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [9]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [10]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [11]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
lgb=LGBMClassifier(n_estimators=200)
lr=LogisticRegression()

In [12]:
train,test=train_test_split(d,test_size=0.3,random_state=123)

columns=list(d.columns)
columns.remove('target')
target='target'
t,v=AST_ecoding(train,test,columns,target,20)

In [13]:
def aucroc1(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC=',auc)
    print('Accuracy=',acc)
    print('Confusion matrix:\n',confusion_matrix(y,pred))
    print('Classification report:\n',classification_report(y,pred))

In [14]:
rem=['RESOURCE_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'MGR_ID_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc',
     'MGR_ID_ROLLUP_12_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'MGR_ID_ROLE_DEPTNAME_enc',
     'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLLUP_12_enc']
lr=LogisticRegression()
t1=t.drop(rem,axis=1)
v1=v.drop(rem,axis=1)
xtr=t1.drop('target',axis=1)
xts=v1.drop(target,axis=1)
ytr=t1[target]
yts=v1[target]
#cols=list(xtr.columns)
mod=lr.fit(xtr,ytr)
aucroc1(mod,xts,yts)

AUC= 0.8825345250838776
Accuracy= 0.9447665547757095
Confusion matrix:
 [[9206   54]
 [ 489   82]]
Classification report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      9260
           1       0.60      0.14      0.23       571

    accuracy                           0.94      9831
   macro avg       0.78      0.57      0.60      9831
weighted avg       0.93      0.94      0.93      9831



In [15]:
xtr.head()

,RESOURCE_enc,ROLE_CODE_enc,RESOURCE_MGR_ID_enc,RESOURCE_ROLE_DEPTNAME_enc,RESOURCE_ROLLUP_12_enc,MGR_ID_ROLE_FAMILY_DESC_enc,MGR_ID_ROLE_CODE_enc,ROLE_FAMILY_DESC_ROLE_CODE_enc
7693,0.052553,0.035277,0.055055,0.052553,0.052553,0.156247,0.046246,0.031866
17676,0.129880,0.036288,0.052553,0.143316,0.129880,0.055055,0.052553,0.039793
1969,0.038539,0.010142,0.055055,0.055055,0.052553,0.050268,0.050268,0.020646
15050,0.055055,0.130292,0.055055,0.055055,0.055055,0.037295,0.033033,0.037295
345,0.092828,0.182512,0.102674,0.089840,0.092828,0.098007,0.098007,0.148434


In [16]:
len(rem)

13

In [17]:
prob=mod.predict_proba(xts)[:,1]
pred=mod.predict(xts)
prob_lr=pd.DataFrame({'Prob_lr':prob,'Pred_lr':pred})
#prob_lgb=pd.read_csv('result_LGB.csv')
#results=pd.concat([prob_lr,prob_lgb],axis=1)
prob_lr.head()


,Prob_lr,Pred_lr
0,0.006249,0
1,0.027647,0
2,0.009888,0
3,0.369689,0
4,0.012442,0


# Kfold AST encoding with Boosting

In [18]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.utils import *
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold
from sklearn.metrics import *


def kfold_AS_target(train,test,columns,target,weight,folds=10):
    
    columns=list(columns)
    kf=KFold()
    t=train
    v=test
    gm=t[target].mean()
    for col in columns:
        
        for i,j in kf.split(t):
            gb=t.loc[t.index[i]].groupby(col)[target].agg(['count','mean'])
            count=gb['count']
            mean=gb['mean']
            gb['smooth']=(count*mean + weight*gm)/(count+weight)
            t.loc[t.index[j],col+'_kenc']=t.loc[t.index[j],col].map(gb['smooth'])
            
        t[col+'_kenc']=t[col+'_kenc'].fillna(gm)
        
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        
        v[col+'_kenc']=v[col].map(gb['smooth'])
        v[col+'_kenc']=v[col+'_kenc'].fillna(gm)
        
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [19]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc

In [20]:
train,test=train_test_split(d,test_size=0.3,random_state=123)
columns=list(train.columns)
columns.remove('target')
target='target'
t,v=kfold_AS_target(train,test,columns,target,100)
train.info()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



<class 'pandas.core.frame.DataFrame'>
Int64Index: 22938 entries, 7693 to 15725
Data columns (total 43 columns):
RESOURCE                               22938 non-null object
MGR_ID                                 22938 non-null object
ROLE_DEPTNAME                          22938 non-null object
ROLE_FAMILY_DESC                       22938 non-null object
ROLE_CODE                              22938 non-null object
target                                 22938 non-null int64
ROLLUP_12                              22938 non-null object
RESOURCE_MGR_ID                        22938 non-null object
RESOURCE_ROLE_DEPTNAME                 22938 non-null object
RESOURCE_ROLE_FAMILY_DESC              22938 non-null object
RESOURCE_ROLE_CODE                     22938 non-null object
RESOURCE_ROLLUP_12                     22938 non-null object
MGR_ID_ROLE_DEPTNAME                   22938 non-null object
MGR_ID_ROLE_FAMILY_DESC                22938 non-null object
MGR_ID_ROLE_CODE                   

In [21]:
#rem=['ROLE_FAMILY_DESC_kenc', 'ROLE_DEPTNAME_ROLLUP_12_kenc', 'ROLE_FAMILY_DESC_ROLE_CODE_kenc', 'MGR_ID_ROLE_FAMILY_DESC_kenc']
rem=[]
xtr=t.drop([target]+rem,axis=1)
ytr=t[target]
xts=v.drop([target]+rem,axis=1)
yts=v[target]

In [22]:
mod=lgb.fit(xtr,ytr)
print('Train:\n')
aucroc1(mod,xtr,ytr)
print('Test:\n')
aucroc1(mod,xts,yts)

Train:

AUC= 0.9969189230383991
Accuracy= 0.984305519225739
Confusion matrix:
 [[21559    53]
 [  307  1019]]
Classification report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     21612
           1       0.95      0.77      0.85      1326

    accuracy                           0.98     22938
   macro avg       0.97      0.88      0.92     22938
weighted avg       0.98      0.98      0.98     22938

Test:

AUC= 0.8802325691352748
Accuracy= 0.9498525073746312
Confusion matrix:
 [[9164   96]
 [ 397  174]]
Classification report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      9260
           1       0.64      0.30      0.41       571

    accuracy                           0.95      9831
   macro avg       0.80      0.65      0.69      9831
weighted avg       0.94      0.95      0.94      9831



In [23]:
pred=mod.predict(xts)
prob=mod.predict_proba(xts)[:,1]
prob_lgb=pd.DataFrame({'Prob_lgb':prob,'Pred_lgb':pred})

In [24]:
result=pd.concat([prob_lr,prob_lgb],axis=1)
result.head()

,Prob_lr,Pred_lr,Prob_lgb,Pred_lgb
0,0.006249,0,0.000214,0
1,0.027647,0,0.018031,0
2,0.009888,0,0.002260,0
3,0.369689,0,0.122152,0
4,0.012442,0,0.000327,0


In [60]:
result['Soft']=(result['Prob_lgb']+result['Prob_lr'])/2
print(roc_auc_score(yts,result['Soft']))
result['Soft_pred']=result['Soft'].apply(lambda x:0 if x<0.5 else 1)
print(classification_report(yts,result['Soft_pred']))
print(confusion_matrix(yts,result['Soft_pred']))

0.8893936786283017
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      9260
           1       0.72      0.18      0.28       571

    accuracy                           0.95      9831
   macro avg       0.83      0.59      0.63      9831
weighted avg       0.94      0.95      0.93      9831

[[9220   40]
 [ 470  101]]


Although f1 score has decreased the auc roc has increase. Which means there is a better chance of finding an optimum threshold